# Noise

In qubit experiments, noise is present in form of environmental influences and
control imperfections. For the specific experiment we are studying, we required
so far the resonance frequency to be constant, which is in reality not the case.
Due to charge and interface dynamics, the resonance frequency randomly changes
with time. We can simulate this behaviour by including the frequency detuning
$\delta_\omega(t)$ as random variable in Schroedinger's equation.

Since the dynamics of the resonance frequency are much slower than our gate
times, we can assume the noise to be quasi static i.e. constant during a single
simulation

\begin{align}
\delta_\omega(t) = \delta_\omega(0) \forall t \in [0, T],
\end{align}

where $T$ is the gate time of a single quantum gate. We assume the detuning to
follow a Gaussian distribution with standard deviation $\sigma_\omega$. To
simplify matters, we can study the idle evolution of the qubit in the rotating
frame, when no driving is applied. The Hamiltonian is then simply:

\begin{align}
H = \delta_\omega \sigma_z
\end{align}


In [1]:
# repeat the setup from the last file

import numpy as np
from qopt import *


sigma_x = DenseOperator.pauli_x()
sigma_y = DenseOperator.pauli_y()
sigma_z = DenseOperator.pauli_z()

zero_matrix = 0 * sigma_x

# we keep the control for the bonus question.
h_ctrl = [.5 * sigma_x, .5 * sigma_y]
h_drift = [zero_matrix]

n_time_steps = 20
total_time = 50  # ns

time_steps = (total_time / n_time_steps) * np.ones((n_time_steps, ))

control_amplitudes = np.zeros((n_time_steps, len(h_ctrl)))
control_amplitudes[:, 0] = .005 * 2 * np.pi  # in 1/ns

We need a new solver to solve the stochastic differential equation as Monte
Carlo experiment. The noise trace generator simulates explicit noise
realizations over which the fidelity can be averaged.

In [2]:

sigma_omega = .001 * 2 * np.pi  # in 1/ns

noise_trace_generator = NTGQuasiStatic(
    standard_deviation=[sigma_omega],
    n_samples_per_trace=n_time_steps,
    n_traces=10,
)

solver = SchroedingerSMonteCarlo(
    h_drift=h_drift * n_time_steps,
    h_ctrl=h_ctrl,
    h_noise=[.5 * sigma_z],
    noise_trace_generator=noise_trace_generator,
    initial_state=DenseOperator(np.eye(2)),
    tau=time_steps
)

solver.set_optimization_parameters(0 * control_amplitudes)

For convenience we will now use the CostFunction class, which gives an interface
to the entanglement in the noise free case and also as average in Monte Carlo
simulations. The infidelity is simply $I = 1 - F$.

In [3]:

noise_infid = OperationNoiseInfidelity(
    solver=solver,
    target=DenseOperator(np.eye(2)),
    neglect_systematic_errors=True
)

noise_infid.costs()

0.015173233873427384

### Analysis Task:
Analytically calculate the entanglement infidelity as function of the standard
deviation $\sigma_\omega$ of the resonance frequency. You may assume the case
of weak or zero driving.

Answer:



### Task:
Plot the averaged entanglement fidelity as function of $\sigma_\omega $.
In the analysis: Fit the data to verify your analytical calculations.

In [1]:
# Solution:


### Task:
Plot the averaged entanglement fidelity as function of total gate time $T$.
In the analysis: Fit the data to verify your analytical calculations.


# Noise Mitigation

Follow the step-by-step instructions to set up the numerical optimization of a
noise mitigating pulse. As solver algorithm, you can use the one initialized
above.

1. Define a cost function for the quantification of systematic errors. Errors
are systematic, if they also occur in the absence of noise. Use the class
OperationInfidelity. As target use an $X_{\pi / 2}$-gate.

2. Initialize an instance of the Simulator class. It shall contain the cost
function for the systematic errors from point 1, and the cost functions for
noise induced errors initialized above.

3. Initialize a LeastSquaresOptimizer. The upper and lower bounds should be
$\pm \, 2 \pi \cdot 0.05$ GHz. You may use the termination conditions defined
below.


4. Use random initial conditions to start the optimizer.
5. Initialize a DataContainer
6. Append the OptimizationResult from the optimization call to the
DataContainer.
7. Use an instance of the Analyser class to plot the values of the cost
function during the optimization.

8. Plot the final pulse on the Bloch sphere.

# Bonus Question

Bonus points if you can explain how the susceptibility to noise is improved.
Hint: Article "Echo Pulse" on Wikipedia and chapter 3.5.2.